In [ ]:
import sqlite3
import pandas as pd
import unicodedata
import ollama
import sys
from rapidfuzz import process
from vanna.chromadb import ChromaDB_VectorStore
from vanna.ollama import Ollama as VannaOllama

# ==============================================================================
# 1. ANALISTA SQL (O Código Robusto que você forneceu)
# ==============================================================================
class SQLAnalyst(ChromaDB_VectorStore, VannaOllama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        VannaOllama.__init__(self, config=config)
        self.bairros = []
        self.entidades = []

    def preparar_agente(self, db_path):
        """Conecta e treina com regras de negócio blindadas."""
        print("   [SQL] Conectando ao banco e carregando metadados...")
        self.connect_to_sqlite(db_path)
        
        try:
            # Extração de metadados reais para o Fuzzy Match
            df_meta = self.run_sql("SELECT DISTINCT bairro, rua, especificacao FROM core_imovel")
            self.bairros = [str(x) for x in df_meta['bairro'].dropna().unique().tolist()]
            ruas = [str(x) for x in df_meta['rua'].dropna().unique().tolist()]
            tipos = [str(x) for x in df_meta['especificacao'].dropna().unique().tolist()]
            self.entidades = self.bairros + ruas + tipos
        except Exception as e:
            print(f"   [SQL] Aviso: Não foi possível carregar metadados ({e}).")

        # Treinamento (Só treina se não tiver dados vetoriais)
        if self.get_training_data().empty:
            print("   [SQL] Realizando treinamento inicial do Vanna...")
            self.train(ddl="""
            CREATE TABLE core_imovel (
                id INTEGER PRIMARY KEY AUTOINCREMENT, 
                titulo VARCHAR(200), 
                descricao TEXT,
                quartos INTEGER, 
                banheiros INTEGER, 
                garagem INTEGER, 
                area DECIMAL, 
                bairro VARCHAR(100), 
                rua VARCHAR(100), 
                preco_aluguel DECIMAL, 
                preco_iptu DECIMAL, 
                preco_condominio DECIMAL, 
                aceita_pets BOOLEAN, 
                especificacao VARCHAR(100) -- apartamento, casa, kitnet, studio, loft, cobertura
            );
            """)

            self.train(documentation=f"""
            - Localização: Juiz de Fora, MG.
            - A coluna de tipo de imóvel se chama 'especificacao'. NUNCA use 'tipo'.
            - REGRA DE PETS: Se o cliente citar 'gato', 'cachorro' ou 'pets', use 'aceita_pets = 1'. 
            - Use LOWER() apenas para colunas de texto: bairro, rua, especificacao.
            - Bairros em JF: {", ".join(self.bairros)}.
            """)

    def normalizar(self, texto):
        nfkd = unicodedata.normalize('NFKD', str(texto))
        return "".join([c for c in nfkd if not unicodedata.combining(c)]).lower().strip()

    def fuzzy_cleanup(self, pergunta):
        """Corrige a pergunta sem duplicar entidades ou alucinar bairros."""
        if not pergunta: return ""
        tokens = pergunta.split()
        resultado = []
        
        for t in tokens:
            t_norm = self.normalizar(t)
            if t_norm.isdigit() or len(t_norm) <= 3:
                resultado.append(t); continue
            
            # Busca correspondência exata ou aproximada
            match = process.extractOne(t_norm, [self.normalizar(e) for e in self.entidades], score_cutoff=88)
            if match:
                idx = [self.normalizar(e) for e in self.entidades].index(match[0])
                entidade_real = self.entidades[idx]
                resultado.append(entidade_real)
            else:
                resultado.append(t)
        
        pergunta_limpa = " ".join(resultado)
        if any(x in pergunta.lower() for x in ["gato", "cachorro", "animal"]):
            pergunta_limpa += " que aceita pets"
            
        return pergunta_limpa

    def executar_consulta(self, pergunta):
        pergunta_limpa = self.fuzzy_cleanup(pergunta)
        print(f"   [SQL] Query Processada: '{pergunta_limpa}'")
        try:
            sql = self.generate_sql(pergunta_limpa)
            
            # Validação simples
            if not sql or "SELECT" not in sql.upper():
                return None, "Não consegui gerar SQL válido."

            df = self.run_sql(sql)
            return df, sql
        except Exception as e:
            return None, f"Erro SQL: {str(e)}"

# ==============================================================================
# 2. BIA PERSONA (A "Boca" do Chatbot)
# ==============================================================================
class BiaPersona:
    def __init__(self, bairros_validos, model_name='llama3.1'):
        self.model = model_name
        self.bairros_validos = bairros_validos
        
    def responder(self, pergunta, df=None, historico=None):
        # Cria o contexto de dados
        if df is not None and not isinstance(df, str) and not df.empty:
            dados_str = df.to_string(index=False)
            contexto = f"RESULTADO DA BUSCA NO BANCO:\n{dados_str}\n(Use estes dados para responder. Se o usuário perguntar detalhes, olhe a tabela.)"
        elif isinstance(df, str):
            contexto = f"AVISO DO SISTEMA: {df}" # Caso de erro
        else:
            # Contexto vazio ou conversa fiada
            contexto = "Nenhum dado de imóvel novo. Apenas converse ou use o histórico."

        system_prompt = f"""
        Você é a Bia, secretária virtual de uma imobiliária em Juiz de Fora.
        
        INSTRUÇÕES:
        1. Se houver imóveis listados em 'RESULTADO DA BUSCA', apresente-os de forma resumida e simpática.
        2. Se o resultado for vazio, diga que não encontrou e sugira bairros: {", ".join(self.bairros_validos[:3])}.
        3. Se for apenas conversa ("Oi", "Obrigado"), seja breve e cordial.
        4. NÃO invente imóveis.
        """
        
        # Constrói o prompt final
        prompt_final = f"{system_prompt}\n\n{contexto}\n\nHistórico recente: {historico}\n\nUsuário: {pergunta}\nBia:"
        
        try:
            response = ollama.generate(model=self.model, prompt=prompt_final, options={'temperature': 0.3})
            return response['response']
        except Exception as e:
            return f"Desculpe, tive um erro técnico: {e}"

# ==============================================================================
# 3. O ORQUESTRADOR (O "Cérebro" que decide)
# ==============================================================================
class Orchestrator:
    def __init__(self, sql_agent, bia_persona):
        self.sql = sql_agent
        self.bia = bia_persona
        self.historico = [] # Memória simples
        self.ultimo_df = None # Memória de dados

    def classificar_intencao(self, texto):
        """
        Usa um modelo rápido para decidir se é SQL (Busca) ou CHAT.
        """
        prompt = f"""
        Classifique a frase do usuário em: BUSCA ou CHAT.
        
        Exemplos:
        "tem apartamento no centro?" -> BUSCA
        "quanto custa o aluguel?" -> BUSCA
        "Oi tudo bem?" -> CHAT
        "Obrigado" -> CHAT
        "Qual o endereço desse aí?" -> CHAT (Pois refere-se ao contexto anterior, não precisa de SQL novo)
        
        Frase: "{texto}"
        Responda APENAS a palavra (BUSCA ou CHAT).
        """
        try:
            # Usando temperatura 0 para ser determinístico
            resp = ollama.generate(model="qwen2.5-coder:7b", prompt=prompt, options={'temperature': 0.0})
            tag = resp['response'].strip().upper()
            if "BUSCA" in tag: return "BUSCA"
            return "CHAT"
        except:
            return "CHAT"

    def processar(self, texto):
        # 1. Identifica Intenção
        intencao = self.classificar_intencao(texto)
        print(f">>> [ROUTER] Intenção: {intencao}")

        dados_para_bia = None

        # 2. Executa Ação
        if intencao == "BUSCA":
            # Passa o texto ORIGINAL para o SQL Analyst (Sem alucinação de '2 quartos')
            df, sql_log = self.sql.executar_consulta(texto)
            self.ultimo_df = df
            dados_para_bia = df
        else:
            # Usa a memória anterior se for conversa sobre o imóvel
            dados_para_bia = self.ultimo_df

        # 3. Gera Resposta Final
        hist_str = "\n".join([f"{h['role']}: {h['content']}" for h in self.historico[-2:]])
        resposta = self.bia.responder(texto, df=dados_para_bia, historico=hist_str)

        # 4. Atualiza Histórico
        self.historico.append({'role': 'user', 'content': texto})
        self.historico.append({'role': 'assistant', 'content': resposta})
        
        return resposta

# ==============================================================================
# EXECUÇÃO PRINCIPAL
# ==============================================================================
if __name__ == "__main__":
    print("\n--- INICIANDO SISTEMA BIA v6 (Router + SQL Analyst Robusto) ---")
    
    # Configurações
    # QwenCoder é ótimo para SQL e Classificação Lógica
    config_sql = {"model": "qwen2.5-coder:7b", "path": "./vanna_chroma_final_v6"}
    
    # 1. Instancia o Especialista SQL (Seu código original)
    analista = SQLAnalyst(config=config_sql)
    analista.preparar_agente("db.sqlite3")
    
    # 2. Instancia a Persona (Llama 3.1 para falar bem)
    bia_persona = BiaPersona(bairros_validos=analista.bairros, model_name='llama3.1:8B')
    
    # 3. Instancia o Cérebro
    bot = Orchestrator(analista, bia_persona)
    
    print("\n✅ Sistema Pronto! (Sem alucinações de parâmetros)")
    
    while True:
        try:
            txt = input("\nVocê: ")
            if txt.lower() in ['sair', 'tchau', 'exit']:
                print("Bia: Tchau! Até logo.")
                break
                
            resp = bot.processar(txt)
            print(f"Bia: {resp}")
            
        except KeyboardInterrupt:
            break


--- INICIANDO SISTEMA BIA v6 (Router + SQL Analyst Robusto) ---
   [SQL] Conectando ao banco e carregando metadados...

✅ Sistema Pronto! (Sem alucinações de parâmetros)
>>> [ROUTER] Intenção: CHAT
Bia: Bom dia! Estou muito bem, obrigada! Como posso ajudar hoje? Você está procurando por um imóvel em Juiz de Fora?
>>> [ROUTER] Intenção: BUSCA
   [SQL] Query Processada: 'Você teria alguma casa no sao matues?'
[{'role': 'system', 'content': "The user provides a question and you provide SQL. You will only respond with SQL code and not with any explanations.\n\nRespond with only SQL code. Do not answer with any explanations -- just the code.\n\nYou may use the following DDL statements as a reference for what tables might be available. Use responses to past questions also to guide you:\n\n\n            CREATE TABLE core_imovel (\n                id INTEGER PRIMARY KEY AUTOINCREMENT, \n                titulo VARCHAR(200), \n                descricao TEXT, \n                quartos INTEGER, \

In [ ]:
import sqlite3
import pandas as pd

# Conecta ao banco de dados
conn = sqlite3.connect('db.sqlite3')

# Carrega os dados da tabela core_imovel para um DataFrame
df = pd.read_sql_query("SELECT * FROM core_imovel", conn)

# Fecha a conexão
conn.close()

# Exibe as primeiras linhas
df.head()